In [1]:
import re
import pandas as pd

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
va_df = pd.read_csv('data/VA-Price-List.csv', skiprows=1)
va_df.columns = va_df.columns.str.lower()

In [3]:
va_df = va_df[va_df.description.str.contains('WHISKY')]
va_df.head()

,description,code,brand,size,age,proof,price
698,CANADIAN WHISKY,11773,Black Velvet,200ml,NaN,80.0,$2.99
699,CANADIAN WHISKY,11774,Black Velvet,375ml,NaN,80.0,$5.59
700,CANADIAN WHISKY,11776,Black Velvet,750ml,NaN,80.0,$9.99
701,CANADIAN WHISKY,11777,Black Velvet,1L,NaN,80.0,$12.99
702,CANADIAN WHISKY,11786,Black Velvet Portable,750ml,NaN,80.0,$9.99


In [12]:
def my_va_transform(s):
    """Function to transform brand
    * make lower
    * remove product types ex: 'bourbon', 'whiskey'
    """
    s = s.lower()
    
    replacements = {"^gentleman jack whiskey$": "jack daniel's gentleman jack",
                    "^pritchard": "prichard",
                    "^balcones baby blue corn whiskey$": "balcones baby blue",
                    "^canadian club rye whisky$": "canadian club",
                    "^catoctin creek roundstone rye whisky$": "catoctin creek roundstone rye",
                    "^e h taylor jr. straight rye$": "colonel e.h. taylor straight rye",
                    "^e h taylor seasoned wood$": "colonel e.h. taylor seasoned wood",
                    "^james e. pepper 1776 rye": "james e. pepper 1776 straight rye",
                    "^lock stock & barrel 16 yr straight rye whiskey$": "lock stock and barrel 16 straight rye",
                    "^michter's us 1 single barrel straight rye$": "michter's us*1",
                    "^michter's us-1 barrel strength rye": "michter's barrel strength rye",
                    "^old overholt$": "old overholt rye",
                    "^wild turkey russell's reserve rye$": "russell's reserve rye 6",
                    "\s7\s": " seven ",
                    "^defiant whisky$": "defiant",
                    "^michter's us1 sour mash$": "michter sour mash",
                    "^red stag": "jim beam red stag",
                    "^four roses 2015 limited edition small batch$": "four roses limited edition 2015",
                    "^four roses 2016 limited edition small batch$": "four roses small batch limited edition 2016",
                    "^i w harper bourbon$": "i.w. harper",
                    "^jesse james bourbon whiskey$": "original jesse james",
                    }
    for k, v in replacements.items():
        s = re.sub(k, v, s)
    
    replace = ['bourbon', 'craft',
               'Year Single Barrel Bourbon',
               '-year Single Barrel Bourbon',
               'whiskey', 'tennessee whiskey', 'tennessee',
               'year', 'yr', '-year single barrel',
               'year single barrel', 'label', "'s",
               'decades', 'whisky', 'yo', '(', ')', 'plastic', 'cask strength', 'scotch'
               ]
    
    for item in replace:
        s = s.replace(item, '')
    
    s = " ".join(s.split()) # remove extra spaces betwen words
    
    return s.strip()

In [13]:
va_df['new_brand'] = va_df['brand'].map(lambda x: my_va_transform(x))
va_df.head()

,description,code,brand,size,age,proof,price,new_brand
698,CANADIAN WHISKY,11773,Black Velvet,200ml,NaN,80.0,$2.99,black velvet
699,CANADIAN WHISKY,11774,Black Velvet,375ml,NaN,80.0,$5.59,black velvet
700,CANADIAN WHISKY,11776,Black Velvet,750ml,NaN,80.0,$9.99,black velvet
701,CANADIAN WHISKY,11777,Black Velvet,1L,NaN,80.0,$12.99,black velvet
702,CANADIAN WHISKY,11786,Black Velvet Portable,750ml,NaN,80.0,$9.99,black velvet portable


In [14]:
wa_df = pd.read_csv('data/Meta-Critic Whisky Database.csv',
                    names=['whisky', 'meta_critic', 'stdev', '#',
                           'cost', 'class', 'super_cluster', 'cluster', 'country', 'type'],
                        skiprows=1)
wa_df.head()

,whisky,meta_critic,stdev,#,cost,class,super_cluster,cluster,country,type
0,Glenfarclas 40yo,9.25,0.30,11,$$$$$+,SingleMalt-like,ABC,A,Scotland,Malt
1,Amrut Greedy Angels (8yo and 10yo),9.20,0.20,6,$$$$$+,SingleMalt-like,ABC,C,India,Malt
2,Redbreast 21yo,9.19,0.32,13,$$$$$,SingleMalt-like,ABC,C,Ireland,Malt
3,Amrut Spectrum,9.18,0.25,8,$$$$$,SingleMalt-like,ABC,C,India,Malt
4,Highland Park 25yo,9.17,0.24,13,$$$$$+,SingleMalt-like,ABC,C,Scotland,Malt


In [15]:
wa_df['whisky'] = wa_df.whisky.str.lower()

In [16]:
wa_df['new_whisky_name'] = wa_df['whisky'].map(lambda x: my_va_transform(x))
wa_df.head()

,whisky,meta_critic,stdev,#,cost,class,super_cluster,cluster,country,type,new_whisky_name
0,glenfarclas 40yo,9.25,0.30,11,$$$$$+,SingleMalt-like,ABC,A,Scotland,Malt,glenfarclas 40
1,amrut greedy angels (8yo and 10yo),9.20,0.20,6,$$$$$+,SingleMalt-like,ABC,C,India,Malt,amrut greedy angels 8 and 10
2,redbreast 21yo,9.19,0.32,13,$$$$$,SingleMalt-like,ABC,C,Ireland,Malt,redbreast 21
3,amrut spectrum,9.18,0.25,8,$$$$$,SingleMalt-like,ABC,C,India,Malt,amrut spectrum
4,highland park 25yo,9.17,0.24,13,$$$$$+,SingleMalt-like,ABC,C,Scotland,Malt,highland park 25


In [17]:
va_new_brand = va_df['new_brand'].tolist()
wa_df_whisky = wa_df['new_whisky_name'].tolist()

In [18]:
exact_matches = set()
for brand in va_new_brand:
    if brand in wa_df_whisky:
        exact_matches.add(brand)
print('Matches: {}'.format(len(exact_matches)))
exact_matches = list(exact_matches)
exact_matches.sort()
print('\n'.join(exact_matches))

Matches: 54
ardbeg auriverdes
ardbeg corryvreckan
ardbeg perpetuum
auchentoshan three wood
balvenie 12 single barrel
bunnahabhain toiteach
canadian mist
caol ila 12
caol ila 30
chivas regal 12
chivas regal 18
cragganmore 25
crown royal
crown royal black
crown royal hand selected barrel
crown royal monarch 75th anniversary
crown royal xo
cutty sark
cutty sark prohibition
dewar white
forty creek barrel select
glenfiddich 12
glenmorangie lasanta
glenmorangie milsean
glenmorangie nectar d'or
glenmorangie quinta ruban
glenmorangie tusail
highland park dark origins
islay mist 8
johnnie walker black
johnnie walker blue
johnnie walker double black
johnnie walker gold reserve
johnnie walker green
johnnie walker platinum
johnnie walker red
lagavulin 12
lagavulin 8
laphroaig 18
laphroaig 25
laphroaig lore
laphroaig quarter cask
laphroaig select
macallan 30 fine oak
macallan edition no. 2
monkey shoulder
oban 18
oban little bay
seagram vo
talisker 18
talisker 25
talisker 30
talisker storm
tomatin 

In [11]:
for brand in va_new_brand:
    matches = process.extract(brand, wa_df_whisky)
    if matches[0][1] != 100:
        print(brand, matches[0])

black velvet ('black velvet 3', 95)
black velvet ('black velvet 3', 95)
black velvet ('black velvet 3', 95)
black velvet ('black velvet 3', 95)
black velvet portable ('jameson select reserve black barrel', 86)
black velvet portable ('jameson select reserve black barrel', 86)
canada house ('canada gold', 70)
canadian club classic small batch ('canadian club classic 12 small batch', 96)
canadian club reserve triple aged ('canadian club reserve 9', 90)
canadian club ('canadian club 30', 95)
canadian club ('canadian club 30', 95)
canadian club ('canadian club 30', 95)
canadian club ('canadian club 30', 95)
canadian hunter traveler ('canadian club 30', 86)
canadian hunter ('canadian rockies 21 batch 1/2', 86)
canadian hunter ('canadian rockies 21 batch 1/2', 86)
canadian hunter ('canadian rockies 21 batch 1/2', 86)
canadian limited ('tomatin cu bocan 1989 limited edition', 86)
canadian limited ('tomatin cu bocan 1989 limited edition', 86)
canadian limited ('tomatin cu bocan 1989 limited edi